 # Classification des avis sur des vêtements de femmes vendus dans le e-commerce

- est ce que les avis que l'on a des vêtements est représentatif de la note qui est attribué ?

idées 
- visualisation de données : quels types de vêtements ont les notes les plus élevées ?
- nb d'avis donné selon l'âge des clients

In [83]:
import pandas as pd

## I. Import des données

Dans un premier temps, nous allons importer nos données. Notre base de données contient des informations sur des avis de vêtements femmes vendus sur internet. Ces données sont issus d'un processus de webscrapping.

In [84]:
data = pd.read_csv("Womens Clothing E-Commerce Reviews.csv", sep = ",")

# renomage première colonne
data = data.rename(columns = {"Unnamed: 0" : "id"})

data.head()

,id,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


## II. Pré-traitement des données

Pour avoir des données plus propres, nous allons effectuer divers pré-traitements.

Pour commencer, nous allon supprimer les lignes où nous avons des valeurs manquantes. 

In [85]:
#supprimer les valeurs manquantes
data = data.dropna()


In [86]:
nb_dpt = data.groupby("Department Name").count()
nb_dpt

,id,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Class Name
Department Name,,,,,,,,,,
Bottoms,3184,3184,3184,3184,3184,3184,3184,3184,3184,3184
Dresses,5371,5371,5371,5371,5371,5371,5371,5371,5371,5371
Intimate,1408,1408,1408,1408,1408,1408,1408,1408,1408,1408
Jackets,879,879,879,879,879,879,879,879,879,879
Tops,8713,8713,8713,8713,8713,8713,8713,8713,8713,8713
Trend,107,107,107,107,107,107,107,107,107,107


### Récupération des données pour notre étude

### Traitement de la casse

- suppression des valeurs manquantes => pas d'avis : inutile
- suppression des caractères spéciaux 
- suppression des majuscules
- suppression des mots vides
- lemmatisation 
- affiche du nombre de mots par étiquette grammaticale
- extraction des mots (groupes de mots) les plus fréquents

- wordcloud

Dans un premier temps, nous allons récupérer les avis.

In [87]:
avis = data["Review Text"]
avis

2        I had such high hopes for this dress and reall...
3        I love, love, love this jumpsuit. it's fun, fl...
4        This shirt is very flattering to all due to th...
5        I love tracy reese dresses, but this one is no...
6        I aded this in my basket at hte last mintue to...
                               ...                        
23481    I was very happy to snag this dress at such a ...
23482    It reminds me of maternity clothes. soft, stre...
23483    This fit well, but the top was very see throug...
23484    I bought this dress for a wedding i have this ...
23485    This dress in a lovely platinum is feminine an...
Name: Review Text, Length: 19662, dtype: object

Nous allons récupérer seulement la partie textuelle de l'avis, cela nous permet de ne avoir un objet Pandas.Series.

In [88]:
from pprint import pprint
liste_avis = data["Review Text"].values.tolist()
pprint(liste_avis[:10])

['I had such high hopes for this dress and really wanted it to work for me. i '
 'initially ordered the petite small (my usual size) but i found this to be '
 'outrageously small. so small in fact that i could not zip it up! i reordered '
 'it in petite medium, which was just ok. overall, the top half was '
 'comfortable and fit nicely, but the bottom half had a very tight under layer '
 'and several somewhat cheap (net) over layers. imo, a major design flaw was '
 'the net over layer sewn directly into the zipper - it c',
 "I love, love, love this jumpsuit. it's fun, flirty, and fabulous! every time "
 'i wear it, i get nothing but great compliments!',
 'This shirt is very flattering to all due to the adjustable front tie. it is '
 'the perfect length to wear with leggings and it is sleeveless so it pairs '
 'well with any cardigan. love this shirt!!!',
 'I love tracy reese dresses, but this one is not for the very petite. i am '
 'just under 5 feet tall and usually wear a 0p in this 

Grâce à cette manipulation, chaque avis est élément d'une liste d'avis. 

Ensuite, nous allons découper les avis en liste de mots et les mettre en minuscules pour pouvoir les analyser plus facilement.

In [89]:
liste_avis_clean = []

for avis in liste_avis : 
    avis = str(avis)
    avis_clean = avis.split()
    avis_clean = avis.lower()
    liste_avis_clean.append(avis_clean)

liste_avis_clean[:10]

['i had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c',
 "i love, love, love this jumpsuit. it's fun, flirty, and fabulous! every time i wear it, i get nothing but great compliments!",
 'this shirt is very flattering to all due to the adjustable front tie. it is the perfect length to wear with leggings and it is sleeveless so it pairs well with any cardigan. love this shirt!!!',
 'i love tracy reese dresses, but this one is not for the very petite. i am just under 5 feet tall and usually wear a 0p in this brand. this dress was very pretty out of

Puis, nous allons tockeniser notre texte. 

In [90]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
liste_avis_clean = [tokenizer.tokenize(str(avis)) for avis in liste_avis_clean]
liste_avis_clean[:10]

[['i',
  'had',
  'such',
  'high',
  'hopes',
  'for',
  'this',
  'dress',
  'and',
  'really',
  'wanted',
  'it',
  'to',
  'work',
  'for',
  'me',
  'i',
  'initially',
  'ordered',
  'the',
  'petite',
  'small',
  'my',
  'usual',
  'size',
  'but',
  'i',
  'found',
  'this',
  'to',
  'be',
  'outrageously',
  'small',
  'so',
  'small',
  'in',
  'fact',
  'that',
  'i',
  'could',
  'not',
  'zip',
  'it',
  'up',
  'i',
  'reordered',
  'it',
  'in',
  'petite',
  'medium',
  'which',
  'was',
  'just',
  'ok',
  'overall',
  'the',
  'top',
  'half',
  'was',
  'comfortable',
  'and',
  'fit',
  'nicely',
  'but',
  'the',
  'bottom',
  'half',
  'had',
  'a',
  'very',
  'tight',
  'under',
  'layer',
  'and',
  'several',
  'somewhat',
  'cheap',
  'net',
  'over',
  'layers',
  'imo',
  'a',
  'major',
  'design',
  'flaw',
  'was',
  'the',
  'net',
  'over',
  'layer',
  'sewn',
  'directly',
  'into',
  'the',
  'zipper',
  'it',
  'c'],
 ['i',
  'love',
  'love',
 

Ensuite, on supprime la ponctuation.

In [91]:
import string
punct = string.punctuation

# Pour chaque token dans chaque avis, si le token n'est pas dans la liste des ponctuations, on le garde
liste_avis_clean = [[token for token in avis if token not in punct] for avis in liste_avis_clean]
liste_avis_clean[:10]

[['i',
  'had',
  'such',
  'high',
  'hopes',
  'for',
  'this',
  'dress',
  'and',
  'really',
  'wanted',
  'it',
  'to',
  'work',
  'for',
  'me',
  'i',
  'initially',
  'ordered',
  'the',
  'petite',
  'small',
  'my',
  'usual',
  'size',
  'but',
  'i',
  'found',
  'this',
  'to',
  'be',
  'outrageously',
  'small',
  'so',
  'small',
  'in',
  'fact',
  'that',
  'i',
  'could',
  'not',
  'zip',
  'it',
  'up',
  'i',
  'reordered',
  'it',
  'in',
  'petite',
  'medium',
  'which',
  'was',
  'just',
  'ok',
  'overall',
  'the',
  'top',
  'half',
  'was',
  'comfortable',
  'and',
  'fit',
  'nicely',
  'but',
  'the',
  'bottom',
  'half',
  'had',
  'a',
  'very',
  'tight',
  'under',
  'layer',
  'and',
  'several',
  'somewhat',
  'cheap',
  'net',
  'over',
  'layers',
  'imo',
  'a',
  'major',
  'design',
  'flaw',
  'was',
  'the',
  'net',
  'over',
  'layer',
  'sewn',
  'directly',
  'into',
  'the',
  'zipper',
  'it',
  'c'],
 ['i',
  'love',
  'love',
 

## III. Classification

### Traitement et séparation des données

Dans cette partie, nous allons chercher à classifier les avis en fonction de leur note. 

Nous allons utiliser la colonne "Rating" comme étiquettes et "Review Text" comme valeurs. 




#### Sélection des informations dans notre dataframe

Nous allons sélectionner les trois colonnes qui vont nous servir pour la classification dans un objectif d'optimiser les temps de calculs et de ne pas avoir d'informations superflus.

In [92]:
# On récupère la colonne id, Rating et Review Text
data = data[["id", "Rating", "Review Text"]]
data.head()

,id,Rating,Review Text
2,2,3,I had such high hopes for this dress and reall...
3,3,5,"I love, love, love this jumpsuit. it's fun, fl..."
4,4,5,This shirt is very flattering to all due to th...
5,5,2,"I love tracy reese dresses, but this one is no..."
6,6,5,I aded this in my basket at hte last mintue to...


#### Analyse de la colonne "Rating"

In [93]:
# Analyse de la colonne "Rating"
data["Rating"].value_counts()

Rating
5    10858
4     4289
3     2464
2     1360
1      691
Name: count, dtype: int64

Nous avons ici des notes allant de 1 à 5. Nous allons diviser ces valeurs en 3 catégories : 
- -1 pour les notes allant de 1 à 2
- 0 pour les notes égales à 3 
- 1 pour les plus élevées (4 et 5)

#### Analyse de la colonne "Review Text"

Notre colonne correspondant aux valeurs est "Review Text". \
Cette colonne contient tous les avis laissés par les internautes sur les différents vêtements.

In [94]:
# ajouter infos d'avant

#### Changement des étiquettes

Pour réaliser notre classification, nous allons donc modifier les étiquettes comme précisé ci-dessus. 

In [95]:
def map_label_to_numeric(label):
    return 1 if label == 5 else 0 if label == 3 or label == 4 else -1

In [96]:
def get_labels(data):
    labels = data[["id","Rating"]]
    labels['Rating'] = labels['Rating'].apply(map_label_to_numeric)
    labels.set_index('id', inplace=True)
    
    # ajouter les labels dans data selon l'id
    data['score_avis'] = labels

    # data['score_avis'] = labels
    return data

In [97]:
data = get_labels(data)
data.head()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_10728\29256686.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels['Rating'] = labels['Rating'].apply(map_label_to_numeric)


,id,Rating,Review Text,score_avis
2,2,3,I had such high hopes for this dress and reall...,0
3,3,5,"I love, love, love this jumpsuit. it's fun, fl...",1
4,4,5,This shirt is very flattering to all due to th...,1
5,5,2,"I love tracy reese dresses, but this one is no...",-1
6,6,5,I aded this in my basket at hte last mintue to...,1


In [98]:
# On analyse la nouevlle colonne "score_avis"
data["score_avis"].value_counts()

score_avis
 1    10858
 0     6753
-1     2051
Name: count, dtype: int64

Grâce à cette manipulation, nous pouvons observer que les avis ayant la note de 5 sont majoritaires dans notre jeu de données. Les avis compris entre 1 et 2 ont une proportion plus faible. 

A présent, nous n'avons plus besoin de la colonne Rating, nous pouvons donc la supprimer du dataframe.

In [105]:
data = data[["id", "Review Text", "score_avis"]]
data.head()

,id,Review Text,score_avis
2,2,I had such high hopes for this dress and reall...,0
3,3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,4,This shirt is very flattering to all due to th...,1
5,5,"I love tracy reese dresses, but this one is no...",-1
6,6,I aded this in my basket at hte last mintue to...,1


#### Division de notre dataframe

Pour réaliser notre classification, nous avons besoin de séparer notre jeu de données en un jeu d'apprentissage, de validation et de test.

Note :
Les données en apprentissage automatique sont généralement séparées en trois jeux :
+ **entraînement** : données destinées à l'apprentissage du modèle ;
+ **validation** : données destinées à une évaluation intermédiaire du modèle pour permettre l'ajustement de ses hyperparamètres. Une fois les hyperparamètres du modèle arrêtés, on peut le ré-entraîner sur l'ensemble des données (entraînement + validation) avant de le tester sur le jeu de test ;
+ **test** : données destinées EXCLUSIVEMENT à l'évaluation FINALE (à réaliser une fois uniquement !) du modèle choisi finalement. Elles ne doivent sous aucune forme servir à la conception du modèle. Il est donc interdit aussi bien de les examiner que d'évaluer le modèle en cours de développement sur ce jeu de données.

In [106]:
# Définir une fonction qui divise notre jeu de données en 3 : train, validation et test
def split_data(data, train_ratio, validation_ratio, test_ratio):
    data_train = data.sample(frac = train_ratio)
    data = data.drop(data_train.index)
    data_validation = data.sample(frac = validation_ratio/(validation_ratio + test_ratio))
    data_test = data.drop(data_validation.index)
    return data_train, data_validation, data_test

# Diviser notre jeu de données en 3 : train, validation et test
data_train, data_validation, data_test = split_data(data, 0.6, 0.2, 0.1)

In [107]:
data_train.shape, data_validation.shape, data_test.shape

((11797, 3), (5243, 3), (2622, 3))

Dans notre cas :
+ entraînement (appelé *Train*) contenant 11797 observations ;
+ validation (appelé *Validation*) contenant 5243 observations ;
+ test (appelé *Test*), contenant 2622 observations, soit environ 22% de la taille du jeu d'entraînement.

In [108]:
data_train.head()

,id,Review Text,score_avis
3744,3744,The second this dress arrived in my store i kn...,1
13430,13430,This dress is comfortable and a great piece to...,0
13996,13996,So comfortable and soft but starting pilling a...,0
2835,2835,I love this thing. i bought it yesterday and a...,1
11553,11553,The sides scooped up a bit too much on me. i w...,-1


In [116]:
data_validation.head()

,id,Review Text,score_avis
9824,9824,I was surprised at how soft this is. it also h...,1
2284,2284,I got some nice things for christmas but this ...,1
19985,19985,I also wanted to love this top but sadly had t...,0
12544,12544,"I tried wearing it, however, i thought i neede...",-1
3532,3532,I love this color blue. when this item went on...,0


In [117]:
data_test.head()

,id,Review Text,score_avis
3,3,"I love, love, love this jumpsuit. it's fun, fl...",1
10,10,Dress runs small esp where the zipper area run...,0
13,13,Bought the black xs to go under the larkspur m...,1
15,15,I took these out of the package and wanted the...,0
18,18,"A flattering, super cozy coat. will work well...",1


### Exploration des données

#### Distribution des classes

Il est important de connaître la répartition des classes dans les données d'entraînement pour pouvoir procéder à notre classification.

In [119]:
# Analyse de la colonne "score_avis" de notre jeu de données d'entrainement
print(data_train["score_avis"].value_counts())

# Calcul des proportions de chaque classe dans notre jeu de données d'entrainement
data_train["score_avis"].value_counts()/len(data_train)

score_avis
 1    6473
 0    4044
-1    1280
Name: count, dtype: int64


score_avis
 1    0.548699
 0    0.342799
-1    0.108502
Name: count, dtype: float64

Nous pouvons observer que les classes de scores sont réparties de manière aléatoire dans notre jeu d'apprentissage. Nous pouvons noter plus de 50% d'avis très favorables, correspondant à la note de 5/5. Les avis négatifs sont en minorité dans notre jeu d'entraînement.

#### Exploration du texte

Pour se faire une idée des textes auxquels nous avons affaire, nous allons les afficher pour savoir quels pré-traitements sont nécessaires.

In [120]:
# Affichage des 5 premiers avis
data_train["Review Text"].values[:5]

array(['The second this dress arrived in my store i knew it was a must have. i am all things bohemian and this dress encompasses everything wonderful about a free fitting, feminine bohemian dress! first off the sleeves are not fitted. they have quite a bit of fabric to them (big armed ladies rejoice!), but have an elastic cuff so you can wear them high or low. the only fitted part to this dress is the lace panel (which still fits true to size), and the slip underneath (which appears to have a bit of str',
       'This dress is comfortable and a great piece to travel with. goes with ballet flats, sandals, to ankle boots.',
       'So comfortable and soft but starting pilling after one wear! sending back.',
       "I love this thing. i bought it yesterday and already am wearing it today, and i have been known to leave things sitting around in my closet for awhile before i wear them, especially if they aren't a sure thing. for reference, i am a curvy, busty size 14. i bought an xl but a l

## Représentation des textes